# Conditioning, stability, and accuracy

Understanding why some numerical calculations go awry. 



## A problematic problem

Problem 5 from the homework asks for the root of $f(x) = x^3 - 2x^2 + 4x/3 - 8/27$. Let's investigate this problem with the secant method (repeated here from the previous lecture). 

In [6]:
function secantsearch(f, a::AbstractFloat, b::AbstractFloat, diagnostics=false) 
# return solution x of f(x) == 0, with a,b serving as initial guesses
    
    fa = f(a)           # cache values of f(a), f(b)     
    fb = f(b)
    
    # set c to better of a,b (and fc, too)
    c,fc = abs(fa) < abs(fb) ? (a,fa) : (b,fb)
    
    floattype = typeof((a+b)/2) # this'll eval to bigger of a,b types
    ϵ = eps(floattype)  # machine epsilon
    n = 1               # iteration count
    N = 1024            # max iterations
    
    # start vector for storing root iterates xₙ, assign x₁ = c
    x = zeros(floattype, 1)
    x[1] = c
    
    # stop when f(c) is small, |b-a| is small relative to a and b
    # or when we reach maximium number of iterations
    while abs(fc) > 10ϵ && abs(b-a)/(abs(a)+abs(b)) > 10ϵ && n < N
    
        # compute new values for c, fc by secant approximation
        c = a - fa*(b-a)/(fb-fa)
        fc = f(c)
        
        # print diagnostics if they're turned on
        diagnostics && println("n = $n, c = $c, f(c) = $(f(c))")
        
        # error and exit if f(c) is bigger than both f(a) and f(b)
        if abs(fc) > abs(fa) && abs(fc) > abs(fb)
            println("secantsearch error: f(c) is bigger than f(a), f(b)")
            break # break out of while loop, don't store bad value c
        end
        
        # replace whichever of a, b has bigger f(a), f(b)
        if abs(fa) > abs(fb)
            a,fa = c,fc
        else
            b,fb = c,fc
        end      
        
        # store c in x, then increment iteration counter
        push!(x,c)   
        n += 1
    end
    
    # return c and vector x
    c, x
    
end

secantsearch (generic function with 2 methods)

In [13]:
f(x) = x.^3 - 2*x.^2 + 4/3*x - 8/27

f (generic function with 1 method)

In [11]:
using Plots
gr()

Plots.GRBackend()

In [20]:
x = linspace(0,1.3)
plot(x,f(x), label="")
plot!(x,0*f(x), color="black", label="")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 -0.2 
 
 
 -0.1 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
<polyline clip-path="url(#clip02)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1436,375.813 50.1879,353.335 61.2322,332.645 72.2765,313.668 83.3208,296.332 94.3652,280.561 105.409,266.282 116.454,253.421 127.498,241.903 138.542,231.654 
 149.587,222.601 160.631,214.669 171.675,207.784 182.72,201.871 193.764,196.858 204.808,192.669 215.853,189.231 226.897,186.47 237.941,184.31 248.986,182.679 
 260.03,181.502 271.074,180.706 282.118,180.215 293.163,179.956 304.207,179.854 315.251,179.837 326.296,179.828 337.34,179.756 348.384,179.544 359.429,179.12 
 370.473,178.408 381.517,177.336 392.562,175.829 403.606,173.812 414.65,171.212 425.695,167.954 436.739,163.965 447.783,159.17 458.828,153.495 469.872,146.867 
 480.916,139.21 491.96,130.451 503.005,120.516 514.049,109.331 525.093,96.821 536.138,82.9126 547.182,67.5315 558.226,50.6036 569.271,32.0548 580.315,11.811 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 39.1436,179.837 50.1879,179.837 61.2322,179.837 72.2765,179.837 83.3208,179.837 94.3652,179.837 105.409,179.837 116.454,179.837 127.498,179.837 138.542,179.837 
 149.587,179.837 160.631,179.837 171.675,179.837 182.72,179.837 193.764,179.837 204.808,179.837 215.853,179.837 226.897,179.837 237.941,179.837 248.986,179.837 
 260.03,179.837 271.074,179.837 282.118,179.837 293.163,179.837 304.207,179.837 315.251,179.837 326.296,179.837 337.34,179.837 348.384,179.837 359.429,179.837 
 370.473,179.837 381.517,179.837 392.562,179.837 403.606,179.837 414.65,179.837 425.695,179.837 436.739,179.837 447.783,179.837 458.828,179.837 469.872,179.837 
 480.916,179.837 491.96,179.837 503.005,179.837 514.049,179.837 525.093,179.837 536.138,179.837 547.182,179.837 558.226,179.837 569.271,179.837 580.315,179.837 
 
 "/>

In [23]:
r,x = secantsearch(f, 0.0, 1.0,true);

n = 1, c = 0.8888888888888891, f(c) = 0.010973936899862813
n = 2, c = 0.8421052631578949, f(c) = 0.005399772129616065
n = 3, c = 0.796785304247992, f(c) = 0.002203020416244428
n = 4, c = 0.7655533526628087, f(c) = 0.0009669710403885645
n = 5, c = 0.7411203540240782, f(c) = 0.0004127229640434882
n = 6, c = 0.7229262275146178, f(c) = 0.000178069284412663
n = 7, c = 0.7091194332075769, f(c) = 7.650996304064961e-5
n = 8, c = 0.6987180509624937, f(c) = 3.292610616567604e-5
n = 9, c = 0.6908601641064995, f(c) = 1.4161066591356075e-5
n = 10, c = 0.6849301974472575, f(c) = 6.091920437989451e-6
n = 11, c = 0.6804532819965489, f(c) = 2.6204324846679228e-6
n = 12, c = 0.6770739084149917, f(c) = 1.1272154389896372e-6
n = 13, c = 0.6745228512285835, f(c) = 4.848808504709368e-7
n = 14, c = 0.6725971277812471, f(c) = 2.085765059556266e-7
n = 15, c = 0.6711434384851674, f(c) = 8.972115939087644e-8
n = 16, c = 0.6700460819689645, f(c) = 3.859443598042134e-8
n = 17, c = 0.6692177117354121, f(c) = 1.6601

Hmmm, that's interesting. The secant method took 36 iterations before stopping on this $f(x)$, when it only took 7 iterations for $f(x) = x^2 - 4$. Also, `c` is still changing in the 5th digit between the last two iterations! It looks like we only have 5 or 6 digits of accuracy in our solution $r \approx 0.66667$. That decimal expansion is pretty suggestive, though. Let's check $f(2/3)$

In [34]:
f(2/3)

0.0

Indeed, the floating-point computation of `f(2/3)` is zero, exactly! So why can't the secant search find the solution, $r=2/3$ to floating-point accuracy? i.e. `r=0.66666666666666667`?

In [51]:
ϵ = logspace(-15,-3,13)

13-element Array{Float64,1}:
 1.0e-15
 1.0e-14
 1.0e-13
 1.0e-12
 1.0e-11
 1.0e-10
 1.0e-9 
 1.0e-8 
 1.0e-7 
 1.0e-6 
 1.0e-5 
 0.0001 
 0.001  

In [49]:
f(2/3+ϵ)

13-element Array{Float64,1}:
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 0.0        
 8.88178e-16
 9.99978e-13
 1.0e-9     